# [모듈 7.1] 리소스 정리

이 노트북은 노트북을 진행하면서 생성한 리소스를 제거합니다. 일부 노트북은 개별 노트북 안에서 리소스를 제거한 것도 있습니다. 여기서는 전체 고급단계에서 진행한 리소스에 대해서 제거를 합니다.

    
---

In [13]:
%store -r project_prefix
%store -r all_mbp_pipeline_name 
%store -r model_package_group_name
%store -r all_deployment_pipeline_name
%store -r train_exp_experiment_name

In [14]:
from src.p_utils import clean_pipeline, delete_endpoint

## 1. 파이프라인 삭제
- 모델 빌딩 파이프라인, 모델 배포 파이프라인을 두개를 삭제 합니다.

In [15]:
# clean_pipeline(pipeline_name = all_adv_pipeline_name, isDeletePipeline=False, verbose=False)   
clean_pipeline(pipeline_name = all_mbp_pipeline_name, isDeletePipeline=True, verbose=False)   
clean_pipeline(pipeline_name = all_deployment_pipeline_name, isDeletePipeline=True, verbose=False)   

pipeline sagemaker-pipeline-phase2-mbp-step-by-step doesn't exists
pipeline sagemaker-pipeline-phase2-deployment-step-by-step exists
pipeline sagemaker-pipeline-phase2-deployment-step-by-step is deleted


## 2. 모델 패키지 삭제
- 해당 패키지 그룹을 삭제 함. 
- 추가적으로 다른 채키지 그룹은 리스트로 보여 줌.

In [8]:
import boto3
sm_client = boto3.Session().client('sagemaker')


def delete_model_package_group(sm_client, ModelPackageGroupName, isDelete=False, verbose=True):
    
    for mpg in sm_client.list_model_package_groups()['ModelPackageGroupSummaryList']:
        print(mpg['ModelPackageGroupName'])

        if (mpg['ModelPackageGroupName'] == ModelPackageGroupName): # 해당 패키지만 그룹만 삭제
            for mp in sm_client.list_model_packages(ModelPackageGroupName=mpg['ModelPackageGroupName'])['ModelPackageSummaryList']:
                print(mp['ModelPackageArn'])
                try:
                    if isDelete:
                        sm_client.delete_model_package(ModelPackageName=mp['ModelPackageArn'])
                        print(f"###### ModelPackageGroupName: {ModelPackageGroupName} is deleted ######")
                except BaseException as error:
                    error_msg = f"Model Package error occurred: {error}"
                    print(error_msg)  
                    pass

            try:
                pass
                if isDelete:
                    sm_client.delete_model_package_group(ModelPackageGroupName=mpg['ModelPackageGroupName'])
            except:
                print("model_pakcage_group error")
                pass
        
#delete_model_package_group(sm_client, ModelPackageGroupName=model_package_group_name, isDelete=False, verbose=True)        
delete_model_package_group(sm_client, ModelPackageGroupName=model_package_group_name, isDelete=True, verbose=True)        

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


sagemaker-pipeline-step-by-step-phase02
arn:aws:sagemaker:ap-northeast-2:057716757052:model-package/sagemaker-pipeline-step-by-step-phase02/3
###### ModelPackageGroupName: sagemaker-pipeline-step-by-step-phase02 is deleted ######
arn:aws:sagemaker:ap-northeast-2:057716757052:model-package/sagemaker-pipeline-step-by-step-phase02/2
###### ModelPackageGroupName: sagemaker-pipeline-step-by-step-phase02 is deleted ######
arn:aws:sagemaker:ap-northeast-2:057716757052:model-package/sagemaker-pipeline-step-by-step-phase02/1
###### ModelPackageGroupName: sagemaker-pipeline-step-by-step-phase02 is deleted ######
tuning-job-model-packages
sagemaker-pipeline-phase2-step-by-step
sagemaker-pipeline-step-by-step
fraud-detect-demo
only-train-template-p-a0vph0bcavzl
demo-project-gsmoon-p-9mvtktb4j47f
scratch3-default-template-p-nszkgtfiasvk
scratch2-default-template-p-t2zokhnjqc6z
scratch2-custom-churn-p-csw7uscei6ov


## 3. Experiment, Trial, Trial Components 삭제

In [9]:
import time
import boto3
sm = boto3.Session().client('sagemaker')
from smexperiments import experiment

def cleanup_experiment(experiment_name, isDelete):
    print("Experiment Name: ", experiment_name)
    trials = sm.list_trials(ExperimentName=experiment_name)['TrialSummaries']
    print('TrialNames:')
    for trial in trials:
        trial_name = trial['TrialName']
        print(f"\n{trial_name} \n")

        components_in_trial = sm.list_trial_components(TrialName=trial_name)
        print('\tTrialComponentNames:')
        for component in components_in_trial['TrialComponentSummaries']:
            component_name = component['TrialComponentName']
            print(f"\t{component_name}")
            sm.disassociate_trial_component(TrialComponentName=component_name, TrialName=trial_name)
            try:
                if isDelete:
                    # comment out to keep trial components
                    sm.delete_trial_component(TrialComponentName=component_name)
                    #print(f"Trial component, {component_name} is deleted")
            except:
                # component is associated with another trial
                continue
            # to prevent throttling
            time.sleep(.5)
        if isDelete:
            sm.delete_trial(TrialName=trial_name)
            print(f"\tTrial_name, {trial_name} is deleted")            
            
    if isDelete:            
        sm.delete_experiment(ExperimentName=experiment_name)
        print(f"\nExperiment {experiment_name} deleted")
    
    
# cleanup_experiment(train_exp_experiment_name, isDelete=False)    
#cleanup_experiment(train_exp_experiment_name, isDelete=True)    

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


## [주의] 아래를 실행하면 현재 생성하신 모든 실험이 삭제 됩니다. 
- 하나의 실험만을 삭제시에는 아래와 같이 사용하세요.

`cleanup_experiment(<experiment_name>, isDelete=True)`

In [ ]:
while True:
    try:
        for exp in experiment.Experiment.list():
            print(exp.experiment_name)
            try:
                cleanup_experiment(exp.experiment_name, isDelete=True)
            except:
                pass
    except BaseException as error:
        error_msg = f"An exception occurred: {error}"
        print(error_msg)  
        break



INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


sagemaker-pipeline-phase2-step-by-step
Experiment Name:  sagemaker-pipeline-phase2-step-by-step
TrialNames:

5tff7trsvtlx 

	TrialComponentNames:
	pipelines-e6svv5i5nej8-fraudeval-yh23d3x2rg-aws-processing-job
	pipelines-e6svv5i5nej8-FraudTrain-70TwTx8fsE-aws-training-job
	pipelines-e6svv5i5nej8-fraudprocess-d1aruu7go3-aws-processing-job
	Trial_name, 5tff7trsvtlx is deleted

e6svv5i5nej8 

	TrialComponentNames:
	e6svv5i5nej8-FraudTu-OKpFf5YtTK-005-68f75397-aws-training-job
	e6svv5i5nej8-FraudTu-OKpFf5YtTK-004-5a6b7984-aws-training-job
	e6svv5i5nej8-FraudTu-OKpFf5YtTK-003-f57f18f3-aws-training-job
	e6svv5i5nej8-FraudTu-OKpFf5YtTK-002-856119f2-aws-training-job
	e6svv5i5nej8-FraudTu-OKpFf5YtTK-001-cc7a1eec-aws-training-job
	pipelines-e6svv5i5nej8-fraudeval-yh23d3x2rg-aws-processing-job
	pipelines-e6svv5i5nej8-FraudTrain-70TwTx8fsE-aws-training-job
	pipelines-e6svv5i5nej8-fraudprocess-d1aruu7go3-aws-processing-job
	Trial_name, e6svv5i5nej8 is deleted

Experiment sagemaker-pipeline-phase2-s

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


	Trial_name, sagemaker-pipeline-step-by-step-phase02-single-train-2021-08-25-12-54-08-623161 is deleted

Experiment sagemaker-pipeline-step-by-step-phase02-single-train deleted


INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials

### Unassigned trial components 삭제
- 필요하면 주석을 제거하고 사용하세요.

In [ ]:
# NextToken=None
# preNextToken = None

# while True:
#     if NextToken:
#         list_trial = sm.list_trial_components(NextToken=NextToken)
#     else:
#         list_trial = sm.list_trial_components()
#     try:
#         NextToken = sm.list_trial_components()['NextToken']
#     except:
#         pass
#     for tc in list_trial['TrialComponentSummaries']:
#         TrialComponentName = tc['TrialComponentName']
#         print(TrialComponentName)
#         try:
#             sm.delete_trial_component(TrialComponentName=TrialComponentName)
#         except:
#             print(f"TrialComponentName : {TrialComponentName}")
#             pass
# #     print(f"NextToken : {NextToken}")
    
#     if preNextToken == NextToken:
#         break
#     preNextToken = NextToken